# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: As described, the problem is a (binary) classification problem. This is because we identify students as either "needs intervention" or "does not need intervention." Presumeably we could expand this to a bigger classification problem to what type of intervention the student needs (e.g. in what subject).**

**Nonetheless, this could be done as a regression probelm to return the probability that someone will fail to graduate without intervention. This could be useful if, for example, there are limited resources for intervention and these need to be given to those it is most likely to help (which could, of course, depend on more than just the probability of failure without intervention).**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


In [2]:
print(student_data.head())
unique_values = [(c, list(student_data[c].unique())) for c in student_data.columns]
for uv in unique_values:
    print(uv)

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3        6   
1  ...        yes       no       5         3      3    1    1      3        4   
2  ...        yes       no       4         3      2    2    3      3       10   
3  ...        yes      yes       3         2      2    1    1      5        2   
4  ...         no       no       4         3      2    1    2      5        4   

  passed  
0     no  
1     no  
2    yes  
3    yes  
4

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)

# TODO: Calculate passing students
n_passed = len(student_data.loc[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data.loc[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/(n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [6]:
# Additional Check: Calculate number of features after pre-processing
n_features_with_dummies = len(X_all.columns)
print "Number of features: {}".format(n_features_with_dummies)

Number of features: 48


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [7]:
# TODO: Import any additional functionality you may need here
import random
random.seed(0)

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
shuffled_indexes = list(range(n_students))
random.shuffle(shuffled_indexes)

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = X_all[:num_train]
X_test = X_all[num_train:]
y_train = y_all[:num_train]
y_test = y_all[num_train:]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print "Training labels has {} samples.".format(y_train.shape[0])
print "Testing labels has {} samples.".format(y_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.
Training labels has 300 samples.
Testing labels has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: Based on the table below, the following three techniques will be used.**

**1. Decision Tree Classifier (DTC)**

**2. K-Nearest Neighbor Classifier (KNN)**

**3. Support Vector Machine Classifier (SVM)**

**These are the three models tend to perform well on data like this. In addition, given my lack of domain knowledge, these three are three models that I can potentially get working well on my own.**

|Model|General Application|Strengths|Weakness|Potentially Good For This Problem?|
|:-:|:-:|:-:|:-:|:-:|
|Decision Tree|Classification and Regression|Good at sussing out complex relationships between the features and the target. Results in an easy to understand structure. Can focus on relevent features and thus get around the curse of dimensionality. Prediction is fast.|Prone to overfitting. Slow to create tree.|Potentially very good. It can select the relvant features and create a good model. However, we will need to be careful to avoid overfitting.|
|Linear/Polynomial Regression|Regression|Good at finding equations to map to continuous data. Simple output. Fast to run once model is built. Easy to understand. Training is relatively fast.|Only good for regression. Cannot capture a wide range of relationships.|No. Not suitable for classification.|
|K-Nearest Neighbor/Kernel Regression|Classification and regression|Simple to understand. Ability to build domain knowledge (via distance metric).|Lots of processing for each prediction. Without good domain knowledge, it treats all features as equally important (especially with normalization). With a lot of features, the curse of dimensionality is really bad for KNN.|Perhaps, but with such little data and so many features and because I'm not a domain expert, it might be hard to get very good results. But its worth a quick check.|
|Neural Networks|Classification and Regression|Theoretically can learn any function.|Complex to get working because of the large number of variations (i.e. the architechture of the neural network). Difficult to explain the inner workings. Can take a lot of training. Because of its power, it is very prone to overfitting.|Probably not. Given the computational limits the school district has, it is unlikely that we can try enough combinations. Also, given how little data there is, the neural network is likely to overfit.|
|Support Vector Machines/Kernel Methods|Classification and Regression|Can do well without a lot of data. Easy to build in domain knowledge. Fairly easy to understand. Tends to generalize well to unseen data. Fitting is low on processing and memory. |Without domain knowledge, if the kernel functions don't work, its hard to do much to make it get good results. Training is high in computation and memory usage.|Very likely to produce good results but maybe hard to get great results.|
|Bayesian Inference|Classification and Regression|Fairly simple model that is easy to explain. Pretty easy to train.|Need to have good domain knowledge to build the belief network.|Maybe not since I don't have good domain knowledge.|
|Boosting|Classification and Regression|Fairly simple to explain. Doesn't overfit. Fairly fast for predicting.|Training can be slow. Need some domain knowledge to come up with the simple rules. Need to have weak learners.|Maybe good, but need to come up with good rules and show there are weak leranrers which maybe too much given the computational restrictions.|

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [9]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import tree
from sklearn import neighbors
from sklearn import svm

# TODO: Initialize the three models
clf_A = tree.DecisionTreeClassifier()
clf_B = neighbors.KNeighborsClassifier()
clf_C = svm.SVC()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for clf in [clf_A, clf_B, clf_C]:
    print(clf)
    print('\n')
    for (X, y) in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        train_predict(clf, X, y, X_test, y_test)
        if clf == clf_A:
            print("Max tree depth: " + str(clf.tree_.max_depth))
        print('\n')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')


Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6557.
Max tree depth: 7


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0026 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.5234.
Max tree depth: 12


Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0038 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.00

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - DTC**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0005           |        0.0004          |     1.0000       |    0.5983       |
| 200               |        0.0007           |        0.0005          |     1.0000       |    0.5691       |
| 300               |        0.0005           |        0.0003          |     1.0000       |    0.6167       |

** Classifer 2 - KNN**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0027              |     0.0025             |     0.8750       |     0.6815      |
| 200               |     0.0074              |     0.0038             |     0.8600       |     0.7761      |
| 300               |     0.0155              |     0.0055             |     0.8597       |     0.7597      |

** Classifer 3 - SVC**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0016              |     0.0015             |     0.8772       |     0.7975      |
| 200               |     0.0061              |     0.0030             |     0.8474       |     0.8026      |
| 300               |     0.0124              |     0.0042             |     0.8686       |     0.7571      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: Selected Model - SVC**

**Based on the above, the SVC is the best model to go forward with. This is for two broad reasons. First, from an accuracy point of view, its seems that the KNN and SVC are better than the DTC. The DTC's are are doing perfect on the training data but poorly on the testing data, a clear indication of overfitting. It is possible that by manipulating the parameters this can be fixed, but as things stand, it doesn't seem like this is the correct approach.**

**Between the KNN and SVC model, I have selected the SVC for two reasons. First, the SVC had the highest F1 score, although its not clear if this is just a fluke or inherent in the model. Second, although they both seem to have comparable run times, in general the KNN model has a longer run time for each prediction since it requires looking at all, or a significant subset of, the data.**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: Training - the support vector classifier works as follows. First, it takes each student and essentially puts them on a big scatter plot. Each student's location on the scatter plot is determined by the their profile i.e. their what school they go to, their sex, their age, the size of their family, etc... (the data we have been given). In addition, each student is marked as either "passed" or "did not pass." Once the students are all plotted on the scatter plot, the SVC will try to place a line separating those students that did pass and those that did not. On one side of the line is the "pass" side and the other is the "did not pass" side. This line can not be perfect, but the algorithm will get a line that minimizes errors and maximizes its ability to generalize on unseen data i.e. predict if a new student will pass or not.**

**Once this line is found, the model predicts if a new student will pass as follows. First, that student is placed on the same scatter plot. Second, the system checks what side ("pass" or "did not pass") the student is on. If on the "pass" side, the system predicts the student will pass. If not, it predicts the student will not pass. If a student is exactly on the line (which is exceedingly unlikely), the system will not have a prediction.**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
print("Setting parameters")
parameters = {
                'criterion' : ['gini', 'entropy'],
                'max_depth' : list(range(5, 21)),
                'min_samples_split' : [pow(2, x) for x in list(range(1, 6))],
                'min_samples_leaf' : [pow(2, x) for x in list(range(1, 5))],
             }

# TODO: Initialize the classifier
print("Initializing classifiers")
clf = tree.DecisionTreeClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
print("Making scorers")
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
print("Initiazling gird search")
grid_obj = GridSearchCV(clf, parameters, verbose=1, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
print("Fitting grid")
grid_obj.fit(X_train, y_train)
print(grid_obj.get_params())

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Setting parameters
Initializing classifiers
Making scorers
Initiazling gird search
Fitting grid
Fitting 3 folds for each of 640 candidates, totalling 1920 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:    1.8s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:    4.0s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:    7.2s
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:   11.2s
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:   16.3s


{'n_jobs': 1, 'verbose': 1, 'estimator__max_features': None, 'estimator__max_depth': None, 'param_grid': {'min_samples_split': [2, 4, 8, 16, 32], 'criterion': ['gini', 'entropy'], 'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'min_samples_leaf': [2, 4, 8, 16]}, 'cv': None, 'scoring': make_scorer(f1_score, pos_label=yes), 'estimator__criterion': 'gini', 'estimator__max_leaf_nodes': None, 'pre_dispatch': '2*n_jobs', 'estimator__min_samples_leaf': 1, 'fit_params': {}, 'estimator__min_samples_split': 2, 'refit': True, 'iid': True, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__presort': False, 'estimator__splitter': 'best', 'estimator__class_weight': None, 'estimator__random_state': None, 'estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None,

[Parallel(n_jobs=1)]: Done 1920 out of 1920 | elapsed:   17.4s finished


In [12]:
print(clf)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=8,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.